In [1]:
import os, sys
import glob 
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import PIL.Image
import PIL.ImageOps

Using TensorFlow backend.


In [2]:
dict_labels = {"black":0, "mg":1, "moth":2, "oil":3}

In [124]:
def label_encoder(name, multi=False):
    global labels
    if multi==True:
        lst = [0,0,0,0]
        for i in range(len(lst)):
            for check in name.split("_")[:-1]:
                if i == dict_labels[check]:
                    lst[i] = 1
        labels.append(lst)
    else:
        if name == "health":
            labels.append([0,0,0,0])
        elif name == "black":
            labels.append([1,0,0,0])
        elif name == "mg":
            labels.append([0,1,0,0])
        elif name == "moth":
            labels.append([0,0,1,0])
        elif name == "oil":
            labels.append([0,0,0,1])

In [125]:
def exif_transpose(img):
    if not img:
        return img

    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img


def load_image_file(file, mode='RGB', size=None):
    # Load the image with PIL
    img = PIL.Image.open(file)
    img = exif_transpose(img)
    img = img.convert(mode)
    if size:
        if type(size) is not tuple:
            print("Wrong type of size")
        else:
            img = img.resize(size)
    return img

In [138]:
def read_dataset(loc, aug=False, input_shape=(224,224)):
    """
    
    """
    global image, labels
    for folders in glob.glob(loc):
        for file in os.listdir(folders):
            if os.path.basename(folders) == "multi":
                label_encoder(file, True)
                img = load_image_file(os.path.join(folders,file), size = (224,224))
                image.append(np.array(img))
            else:
                label_encoder(os.path.basename(folders), False)
                img = load_image_file(os.path.join(folders,file), size = (224,224))
                image.append(np.array(img))
    
    x = np.array(img, dtype='float32') / 255.0
    y = np.array(labels, dtype='float32')
    return x, y

In [139]:
image, labels = [],[]
read_dataset("./new/new_train_b_tmp/*")
labels

[[1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1]]

In [141]:
x_train = np.array(train, dtype='float32') / 255.0
y_train = np.array(labels, dtype='float32')

print(x_train.shape)
y_train

(28, 224, 224, 3)


array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [1., 1., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [52]:
# append encoder to a list for each sample
a = []
b = []
a.append(0)
a.append(1)
a.append(0)
a.append(0)
b.append(a)
b.append(a)
np.array(b)

array([[0, 1, 0, 0],
       [0, 1, 0, 0]])

In [95]:
# split img filename when folder is multi-label
img = "black_1.jpg"
img2 = "black_mg_1.jpg"
img3 = "mg_oil_moth_2.jpg"
dict_labels = {"black":0, "mg":1, "moth":2, "oil":3}
print(img.split("_")[:-1])
print(img3.split("_")[:-1])
#for check in img3.split("_")[:-1]:
    #print(check)
    #print(dict_labels[check])

lst = [0,0,0,0]
for i in range(len(lst)):
    for check in img3.split("_")[:-1]:
        if i == dict_labels[check]:
            print(lst[i])
            lst[i] = 1

lst
#label_encoder(img.split("_")[:-1], True)
#label_encoder(img2.split("_")[:-1], True)
#label_encoder(img3.split("_")[:-1], True)

['black']
['mg', 'oil', 'moth']
0
0
0


[0, 1, 1, 1]

In [119]:
# load image
img = load_image_file("black.jpg", size = (224,224))
print(np.array(img))

[[[128 146 122]
  [137 154 136]
  [150 164 151]
  ...
  [117 131  82]
  [112 126  77]
  [127 142  87]]

 [[120 141 110]
  [130 148 126]
  [147 163 150]
  ...
  [127 140  86]
  [113 132  77]
  [113 136  82]]

 [[119 140 107]
  [135 153 129]
  [148 162 149]
  ...
  [133 154  89]
  [135 153  91]
  [110 129  74]]

 ...

 [[255 236 232]
  [254 239 232]
  [254 240 231]
  ...
  [115 118  65]
  [134 144  75]
  [144 167  87]]

 [[255 241 234]
  [255 242 236]
  [254 244 234]
  ...
  [125 149  75]
  [141 167  80]
  [156 189 102]]

 [[255 237 233]
  [254 241 235]
  [253 243 234]
  ...
  [143 170  89]
  [150 182  99]
  [155 191 104]]]


In [31]:
labels_hot=[]
dict_labels = {"black":"0", "healthy":"1", "mg":"mg", "moth":"moth", "oil":"oil"}
labels_hot.append(dict_labels["mg"])
labels_hot.append(dict_labels["oil"])
labels_hot.append(dict_labels["moth"])

In [32]:
labels_hot

['mg', 'oil', 'moth']

In [33]:
y = to_categorical(LabelEncoder().fit_transform(labels_hot))
y

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)